In [1]:
# import the dependencies

import os
import pandas as pd

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship


In [2]:
# Pull in our GCHN_archive class

In [3]:
import GHCN_archive as GHCN

In [4]:
ghcn = GHCN.GHCN_archive()

Checking: static\src\ghcnd_all.tar.gz
Checking: static\src\ghcnd-version.txt
Checking: static\src\ghcnd-stations.txt
Checking: static\src\ghcnd-inventory.txt
Checking: static\src\ghcnd-countries.txt
Checking: static\src\ghcnd-states.txt
Version found: 3.27-upd-2019081018


In [5]:
ghcn.list_files()

ghcnd-countries.txt
ghcnd-inventory.txt
ghcnd-states.txt
ghcnd-stations.txt
ghcnd-version.txt
ghcnd_all
ghcnd_all.tar.gz
US1MNAA0002.dly
USW00014925.dly
USW00014992.dly
USW00094960.dly


In [6]:
ghcn.files

{'GHCN_TGZ': 'ghcnd_all.tar.gz',
 'GHCN_VERSION': 'ghcnd-version.txt',
 'GHCN_STATIONS': 'ghcnd-stations.txt',
 'GHCN_INVENTORY': 'ghcnd-inventory.txt',
 'GHCN_COUNTRIES': 'ghcnd-countries.txt',
 'GHCN_STATES': 'ghcnd-states.txt',
 'GHCN_DAILY': 'static\\src\\ghcnd_all'}

In [7]:
ghcn.path

'static\\src'

In [8]:
# How to access the files of our archive
print(os.path.join(ghcn.path, ghcn.files['GHCN_INVENTORY']))

static\src\ghcnd-inventory.txt


In [9]:
# Create the database directory
DATABASE_DIRECTORY = os.path.join("static", "data")
DATABASE_FILE = "ghcn.db"
if not os.path.exists(DATABASE_DIRECTORY):
    os.makedirs(DATABASE_DIRECTORY)
    
DATABASE_PATH = os.path.join(DATABASE_DIRECTORY, DATABASE_FILE)

In [10]:
# Create a connection to a SQLite database
engine = create_engine(f'sqlite:///{DATABASE_PATH}')

In [11]:
print(engine)

Engine(sqlite:///static\data\ghcn.db)


# Declare the classes 

In [12]:
class Countries(Base):
    __tablename__ = 'countries'
    fips = Column(String, primary_key=True)
    name = Column(String)
    
class States(Base):
    __tablename__ = 'states'
    fips = Column(String, primary_key=True)
    name = Column(String)

class Stations(Base):
    __tablename__ = 'stations'
    station_id = Column(String, primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    country_id = Column(String, ForeignKey('countries.fips'))
    state_id = Column(String, ForeignKey('states.fips'))
    name = Column(String)
    gsn = Column(String)
    hcn = Column(String)
    wmo = Column(Integer)
    
    country = relationship('Countries', foreign_keys=[country_id])
    state = relationship('States', foreign_keys=[state_id])

# Create the Inventory class
class Inventory(Base):
    __tablename__ = 'inventory'
    id = Column(Integer, primary_key=True)
    station_id = Column(String(12), ForeignKey('stations.station_id'))
    element = Column(String(6))
    first_year = Column(Integer)
    last_year = Column(Integer)
    
    station = relationship('Stations', foreign_keys=[station_id])
    
    


In [13]:
# Create the four tables within the database
Base.metadata.create_all(engine)

# First fill the Countries table

In [14]:
countries_file = os.path.join(ghcn.path, ghcn.files['GHCN_COUNTRIES'])
countries_colspecs = [(0, 3), (3, 50)]
countries_colnames = ['fips', 'name']

countries_df = pd.read_fwf(countries_file, colspecs=countries_colspecs,
                          names=countries_colnames)
countries_df.head()

,fips,name
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan


In [15]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [16]:
# Iterate over the rows and add to the table
for row in countries_df.itertuples():
    country = Countries(fips=row.fips, name=row.name)
    session.add(country)
    session.commit()

In [18]:
items = session.query(Countries)
for item in items:
    print(f"fips: {item.fips}, name: {item.name}")

fips: AC, name: Antigua and Barbuda
fips: AE, name: United Arab Emirates
fips: AF, name: Afghanistan
fips: AG, name: Algeria
fips: AJ, name: Azerbaijan
fips: AL, name: Albania
fips: AM, name: Armenia
fips: AO, name: Angola
fips: AQ, name: American Samoa [United States]
fips: AR, name: Argentina
fips: AS, name: Australia
fips: AU, name: Austria
fips: AY, name: Antarctica
fips: BA, name: Bahrain
fips: BB, name: Barbados
fips: BC, name: Botswana
fips: BD, name: Bermuda [United Kingdom]
fips: BE, name: Belgium
fips: BF, name: Bahamas, The
fips: BG, name: Bangladesh
fips: BH, name: Belize
fips: BK, name: Bosnia and Herzegovina
fips: BL, name: Bolivia
fips: BM, name: Burma
fips: BN, name: Benin
fips: BO, name: Belarus
fips: BP, name: Solomon Islands
fips: BR, name: Brazil
fips: BU, name: Bulgaria
fips: BX, name: Brunei
fips: BY, name: Burundi
fips: CA, name: Canada
fips: CB, name: Cambodia
fips: CD, name: Chad
fips: CE, name: Sri Lanka
fips: CF, name: Congo (Brazzaville)
fips: CG, name: Cong

# Fill the States Table

In [19]:
states_file = os.path.join(ghcn.path, ghcn.files['GHCN_STATES'])
states_colspecs = [(0, 3), (3, 50)]
states_colnames = ['fips', 'name']

states_df = pd.read_fwf(states_file, colspecs=states_colspecs,
                          names=states_colnames)
states_df.head()

,fips,name
0,AB,ALBERTA
1,AK,ALASKA
2,AL,ALABAMA
3,AR,ARKANSAS
4,AS,AMERICAN SAMOA


In [20]:
# Iterate over the rows and add to the table
for row in states_df.itertuples():
    state = States(fips=row.fips, name=row.name)
    session.add(state)
    session.commit()

In [21]:
items = session.query(States)
for item in items:
    print(f"fips: {item.fips}, name: {item.name}")

fips: AB, name: ALBERTA
fips: AK, name: ALASKA
fips: AL, name: ALABAMA
fips: AR, name: ARKANSAS
fips: AS, name: AMERICAN SAMOA
fips: AZ, name: ARIZONA
fips: BC, name: BRITISH COLUMBIA
fips: CA, name: CALIFORNIA
fips: CO, name: COLORADO
fips: CT, name: CONNECTICUT
fips: DC, name: DISTRICT OF COLUMBIA
fips: DE, name: DELAWARE
fips: FL, name: FLORIDA
fips: FM, name: MICRONESIA
fips: GA, name: GEORGIA
fips: GU, name: GUAM
fips: HI, name: HAWAII
fips: IA, name: IOWA
fips: ID, name: IDAHO
fips: IL, name: ILLINOIS
fips: IN, name: INDIANA
fips: KS, name: KANSAS
fips: KY, name: KENTUCKY
fips: LA, name: LOUISIANA
fips: MA, name: MASSACHUSETTS
fips: MB, name: MANITOBA
fips: MD, name: MARYLAND
fips: ME, name: MAINE
fips: MH, name: MARSHALL ISLANDS
fips: MI, name: MICHIGAN
fips: MN, name: MINNESOTA
fips: MO, name: MISSOURI
fips: MP, name: NORTHERN MARIANA ISLANDS
fips: MS, name: MISSISSIPPI
fips: MT, name: MONTANA
fips: NB, name: NEW BRUNSWICK
fips: NC, name: NORTH CAROLINA
fips: ND, name: NORTH DA

# Fill the Stations Table

In [23]:
stations_file = os.path.join(ghcn.path, ghcn.files['GHCN_STATIONS'])
stations_colspecs = [(0,12),(12,21),(21,31),(31,38),(38,41),(41,72),(72,76),(76,80),(80,86)]
stations_colnames = ["station_id", "latitude", "longitude", "elevation", "state", "name", "gsn_flag", "hcn_crn_flag", "wmo_id"]

stations_df = pd.read_fwf(stations_file, colspecs=stations_colspecs,
                          names=stations_colnames)
stations_df.head()

,station_id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0


In [24]:
def country_id(station_id):
    return station_id[:2]

stations_df['country'] = stations_df['station_id'].map(country_id)
stations_df.head()

,station_id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id,country
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,AC
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN,AC
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0,AE
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0,AE
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0,AE


In [26]:
# Iterate over the rows and add to the table
for row in stations_df.itertuples():
    station = Stations(station_id = row.station_id,
                       latitude = row.latitude,
                       longitude = row.longitude,
                       elevation = row.elevation,
                       country_id = row.country,
                       state_id = row.state,
                       name = row.name,
                       gsn = row.gsn_flag,
                       hcn = row.hcn_crn_flag,
                       wmo = row.wmo_id)
    session.add(station)
    session.commit()

In [29]:
items = session.query(Stations.station_id, Stations.latitude, Stations.longitude, Stations.name).limit(5).all()
for item in items:
    print(item)

('ACW00011604', 17.1167, -61.7833, 'ST JOHNS COOLIDGE FLD')
('ACW00011647', 17.1333, -61.7833, 'ST JOHNS')
('AE000041196', 25.333, 55.517, 'SHARJAH INTER. AIRP')
('AEM00041194', 25.255, 55.364, 'DUBAI INTL')
('AEM00041217', 24.433, 54.651, 'ABU DHABI INTL')


In [31]:
items = session.query(Stations.station_id, Stations.latitude, Stations.longitude, Stations.name).filter(Stations.state_id == 'MN').all()
print(f"The number of stations in Minnesota: {len(items)}")
for item in items:
    print(item)

The number of stations in Minnesota: 1251
('US1MNAA0002', 45.1947, -93.3257, 'COON RAPIDS 1.5 NNW')
('US1MNAA0003', 45.1962, -93.2755, 'COON RAPIDS 2.2 NE')
('US1MNAA0004', 45.409, -93.309, 'SAINT FRANCIS 4.0 E')
('US1MNAA0005', 45.3846, -93.1553, 'EAST BETHEL 3.1 NE')
('US1MNAA0006', 45.1708, -93.257, 'BLAINE 2.4 W')
('US1MNAA0012', 45.2451, -93.2068, 'HAM LAKE 0.7 SSW')
('US1MNAA0013', 45.1648, -93.2855, 'COON RAPIDS 1.4 ESE')
('US1MNAA0014', 45.2183, -93.2973, 'ANDOVER 3.0 SSE')
('US1MNAA0015', 45.1426, -93.1195, 'LINO LAKES 2.5 SW')
('US1MNAA0016', 45.3712, -93.2115, 'EAST BETHEL 1.1 NNW')
('US1MNAA0017', 45.0598, -93.2408, 'COLUMBIA HEIGHTS 0.9 NNE')
('US1MNAA0018', 45.2854, -93.3438, 'ANDOVER 2.1 NNW')
('US1MNAA0019', 45.189, -93.3719, 'COON RAPIDS 3.2 WNW')
('US1MNAA0020', 45.1646, -93.2072, 'BLAINE 0.4 S')
('US1MNAA0021', 45.2657, -93.2221, 'HAM LAKE 1.2 NW')
('US1MNAA0022', 45.3934, -93.365, 'SAINT FRANCIS 1.3 ESE')
('US1MNAA0023', 45.0847, -93.2648, 'FRIDLEY 0.3 W')
('US1MNAA

# Load the Inventory Table

In [33]:
inventory_file = os.path.join(ghcn.path, ghcn.files['GHCN_INVENTORY'])
inventory_colspecs = [(0,12), (12,21), (21, 31), (31,36), (36,41), (41,45)]
inventory_colnames = ["station_id", "latitude", "longitude", "element", "first_year", "last_year"]
inventory_df = pd.read_fwf(inventory_file, colspecs=inventory_colspecs, header=None,
                          names=inventory_colnames)
inventory_df.head()

,station_id,latitude,longitude,element,first_year,last_year
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949


In [34]:
# Save our work to this point in .csv files
stations_df.to_csv(os.path.join(ghcn.path, "stations.csv"))
inventory_df.to_csv(os.path.join(ghcn.path, "inventory.csv"))

In [35]:
# Iterate over the rows and add to the table
for row in inventory_df.itertuples():
    inventory = Inventory(station_id = row.station_id,
                          element = row.element,
                          first_year = row.first_year,
                          last_year = row.last_year)
    session.add(inventory)
    session.commit()

# Query the New Database

In [36]:
items = session.query(Inventory.station_id, Inventory.element, Inventory.first_year, Inventory.last_year).limit(5).all()
for item in items:
    print(item)

('ACW00011604', 'TMAX', 1949, 1949)
('ACW00011604', 'TMIN', 1949, 1949)
('ACW00011604', 'PRCP', 1949, 1949)
('ACW00011604', 'SNOW', 1949, 1949)
('ACW00011604', 'SNWD', 1949, 1949)


# Pass Through Queries - SQL statements passed through to the database

In [53]:
sql_query = """
SELECT station_id, first_year, last_year 
FROM inventory
WHERE element == 'SNWD'
ORDER By station_id
"""
query_return = pd.read_sql_query(sql_query, engine)
print(query_return.shape)
query_return.head()

(58166, 3)


,station_id,first_year,last_year
0,ACW00011604,1949,1949
1,ACW00011647,1957,1970
2,AF000040930,1982,1992
3,AFM00040938,1982,2019
4,AFM00040948,1974,2018


In [55]:
sql_query = """
SELECT * FROM stations
WHERE country_id = 'US'
"""
query_return = pd.read_sql_query(sql_query, engine)
print(query_return.shape)
query_return.head()

(60787, 10)


,station_id,latitude,longitude,elevation,country_id,state_id,name,gsn,hcn,wmo
0,US009052008,43.7333,-96.6333,482.0,US,SD,SIOUX FALLS (ENVIRON. CANADA),None,None,NaN
1,US10RMHS145,40.5268,-105.1113,1569.1,US,CO,RMHS 1.6 SSW,None,None,NaN
2,US10adam001,40.5680,-98.5069,598.0,US,NE,JUNIATA 1.5 S,None,None,NaN
3,US10adam002,40.5093,-98.5493,601.1,US,NE,JUNIATA 6.0 SSW,None,None,NaN
4,US10adam003,40.4663,-98.6537,615.1,US,NE,HOLSTEIN 0.1 NW,None,None,NaN


In [61]:
def make_sql_query_string(element, first_year, last_year):
    sql_query = f"SELECT station_id FROM inventory WHERE element = {element} AND first_year >= {first_year} AND last_year <= {last_year}"
    return sql_query

In [64]:
make_sql_query_string("'SNWD'", 1949, 2019)

"SELECT station_id FROM inventory WHERE element = 'SNWD' AND first_year >= 1949 AND last_year <= 2019"

In [66]:
query_return = pd.read_sql_query(make_sql_query_string("'PRCP'", 1970, 2019), engine)
print(query_return.shape)
query_return.head()

(56547, 1)


,station_id
0,AEM00041194
1,AEM00041217
2,AEM00041218
3,AF000040930
4,AFM00040938


In [80]:
from sqlalchemy import and_
t = session.query(
    Inventory.station_id
).filter(Inventory.first_year <= 2010).filter(Inventory.last_year == 2019).filter(Inventory.element == 'SNWD').subquery('t')

query = session.query(Stations).filter(and_(
    Stations.country_id == 'US',
    Stations.state_id == 'MN',
    Stations.station_id == t.c.station_id
)
)

print(query.count())
for item in query:
    print(item.station_id, item.name, item.state.name)

179
US1MNAA0004 SAINT FRANCIS 4.0 E MINNESOTA
US1MNBW0001 SPRINGFIELD 1.7 NNW MINNESOTA
US1MNCN0001 CARLTON 5.6 NE MINNESOTA
US1MNCP0001 MONTEVIDEO 0.4 SSW MINNESOTA
US1MNCS0001 PILLAGER 5.1 N MINNESOTA
US1MNCV0001 CARVER 0.7 W MINNESOTA
US1MNCY0001 MOORHEAD 2.0 N MINNESOTA
US1MNDK0014 INVER GROVE HEIGHTS 1.3 ENE MINNESOTA
US1MNDK0015 BURNSVILLE 3.0 NE MINNESOTA
US1MNGH0003 LAKE CITY 1.2 NNW MINNESOTA
US1MNHN0006 LONG LAKE 0.2 WSW MINNESOTA
US1MNHN0009 MINNEAPOLIS 3.0 NNW MINNESOTA
US1MNHN0014 MINNETRISTA 1.5 SSE MINNESOTA
US1MNHN0017 EDEN PRAIRIE 3.3 WSW MINNESOTA
US1MNHN0019 EDINA 1.3 SW MINNESOTA
US1MNHN0022 MINNEAPOLIS 3.3 SW MINNESOTA
US1MNKB0003 BROOK PARK 9.0 WNW MINNESOTA
US1MNKD0001 BELGRADE 8.0 ESE MINNESOTA
US1MNLY0002 MARSHALL 0.6 S MINNESOTA
US1MNOT0003 BATTLE LAKE 2S MINNESOTA
US1MNPK0002 ERSKINE 7.6 SW MINNESOTA
US1MNRC0001 MONTGOMERY 3.2 ENE MINNESOTA
US1MNRM0002 NORTH SAINT PAUL 1.0 NNW MINNESOTA
US1MNRM0005 ROSEVILLE 1.4 SSE MINNESOTA
US1MNRM0015 ROSEVILLE 2.1 NW MINN

In [ ]:
def find_stations_with(session, element, first_year=None, last_year=None):
    """
    Args:  
        element: one of the elements in the inventory table.
        first_year: the earliest year that must be included.  If None, we do not filter on first_year
        last_year: the latest year that must be included.  If None, we do not filter on last_year
    """
    
    if first_year:
        if last_year: 
            t = session.query(
                Inventory.station_id,
                Inventory.first_year,
                Inventory.last_year
            ).filter(Inventory.first_year <= first_year)\
            .filter(Inventory.last_year >= last_year)\
            .filter(Inventory.element == element).subquery('t')
        else:
            t = session.query(
                Inventory.station_id,
                Inventory.first_year, 
                Inventory.last_year
            ).filter(Inventory.first_year <= first_year)\
            .filter(Inventory.element == element).subquery('t')
    else:
        if last_year: 
            t = session.query(
                Inventory.station_id,
                Inventory.first_year, 
                Inventory.last_year
            ).filter(Inventory.last_year >= last_year)\
            .filter(Inventory.element == element).subquery('t')
        else:
            t = session.query(
                Inventory.station_id,
                Inventory.first_year, 
                Inventory.last_year
            ).filter(Inventory.element == element).subquery('t')
        

    query = session.query(Stations).filter(
        Stations.station_id == t.c.station_id
    )

    return pd.read_sql(query.statement, session.bind)


In [ ]:
rain_1970_1980_df = find_stations_with(session, 'PRCP', 1970, 1980)
print(rain_1970_1980_df.shape)
rain_1970_1980_df.head()

In [ ]:
any_rain_df = find_stations_with(session, 'PRCP')
print(any_rain_df.shape)
any_rain_df.head()